In [ ]:
import pandas as pd
import numpy as np

In [ ]:
def get_twse_price(date=None, filename=None):
    import requests
    """抓取TWSE台灣證交所上市公司收盤價格
    Parameters
    ----------
    date : (str) 收盤日期 (format: yyyymmdd)
    filename : the file for storing stock prices
    """
    url = 'http://www.twse.com.tw/exchangeReport/MI_INDEX'
    values = {
        'response': 'csv',
        'date': date,
        'type': 'ALLBUT0999',
    }
    
    if date is None:
        print('Warning: Missing date')
        return
    
    if filename is None:
        filename = 'stock_price.csv'
    try:
        print('Downloading stock prices...')
        r = requests.post(url, values)
        with open(filename, 'w', encoding='utf-8') as f:
            f.write(r.text)
    except:
        print('Warning: Fail to get stock prices')
    else:
        print('Status: stock prices stored in %s' % filename)

In [ ]:
# get_twse_price('20181203')

In [ ]:
columns = ["證券代號","證券名稱","成交股數","成交筆數","成交金額","開盤價","最高價","最低價","收盤價","漲跌(+/-)","漲跌價差","最後揭示買價","最後揭示買量","最後揭示賣價","最後揭示賣量","本益比"]
df = pd.read_csv('stock_price.csv',
    header=None, names=columns, index_col=False, 
    thousands=',', quotechar= '"', skiprows=lambda x: x <= 163, encoding='utf-8')

print(df.shape)
df.head()

In [ ]:
df = df[df["證券代號"].apply(lambda x: len(x) == 4)]
print(df.shape)
df.head()

In [ ]:
df = df[["證券代號","證券名稱","成交股數","開盤價","最高價","最低價","收盤價"]]
df.reset_index(drop=True, inplace=True)
df.set_index('證券代號', inplace=True)
df.head()

In [ ]:
# "Thousands Separators" make price data "str", not "float" numbers
print(df.loc['3008'])
print(type(df.loc['3008', '收盤價']))

#### Series.map(arg, na_action=None)
Map values of <font color='red'>Series</font> using input correspondence (a dict, Series, or function).

In [ ]:
def f(x):
    if ',' in x:
        x = x.replace(',', '')
    try:
        return float(x)
    except:
        return None

df['開盤價'] = df['開盤價'].map(f)
df['最高價'] = df['最高價'].map(f)
df['最低價'] = df['最低價'].map(f)
df['收盤價'] = df['收盤價'].map(f)

print(df.loc['3008'])
print(type(df.loc['3008', '收盤價']))

In [ ]:
df['當日漲幅(%)'] = round((df['收盤價'] - df['開盤價']) / df['開盤價'] * 100, 2)
df.head()

In [ ]:
df1 = df.sort_values(by='當日漲幅(%)', ascending=False)
df1.head()

In [ ]:
print(f'Max. 成交股數 = {df["成交股數"].max():.0f}')
print(f'Avg. 成交股數 = {df["成交股數"].mean():.0f}')
print(f'Min. 成交股數 = {df["成交股數"].min():.0f}')

In [ ]:
print(f'當日成交最熱絡: {df["證券名稱"][df["成交股數"].idxmax()]}')
df[df['證券名稱'] == '旺宏']

In [ ]:
df.loc['2454']